In [2]:
import pandas as pd
import re

In [3]:
recipe_df = pd.read_parquet('./resource/recipes.parquet')

In [15]:
cleaned_df = recipe_df
cleaned_df['RecipeId'] = cleaned_df['RecipeId'].astype(int)
cleaned_df['CookTime'] = cleaned_df['CookTime'].fillna('')
cleaned_df['AggregatedRating'] = cleaned_df['AggregatedRating'].fillna(0)
cleaned_df['ReviewCount'] = cleaned_df['ReviewCount'].fillna(0)
cleaned_df['Description'] = cleaned_df['Description'].fillna('')
cleaned_df['RecipeCategory'] = cleaned_df['RecipeCategory'].fillna('')
cleaned_df['RecipeServings'] = cleaned_df['RecipeServings'].fillna(0)
cleaned_df['RecipeYield'] = cleaned_df['RecipeYield'].fillna('0')
for row in cleaned_df.loc[cleaned_df['Images'].isna()].index:
    cleaned_df.at[row, 'Images'] = []
cleaned_df.isna().sum()

cleaned_df

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,id
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09 21:46:00+00:00,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,...,8.0,29.8,37.1,3.6,30.2,3.2,4.0,0,"[Toss 2 cups berries with sugar., Let stand fo...",None
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29 13:12:00+00:00,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,...,372.8,368.4,84.4,9.0,20.4,63.4,6.0,0,[Soak saffron in warm milk for 5 minutes and p...,None
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05 19:52:00+00:00,This is from one of my first Good House Keepi...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.0,1.8,81.5,0.4,77.2,0.3,4.0,0,"[Into a 1 quart Jar with tight fitting lid, pu...",None
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03 14:54:00+00:00,This dish is best prepared a day in advance to...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"[Drain the tofu, carefully squeezing out exces...",None
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19 06:19:00+00:00,Make and share this Cabbage Soup recipe from F...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.0,959.3,25.1,4.8,17.7,4.3,4.0,0,"[Mix everything together and bring to a boil.,...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522512,541379,Meg's Fresh Ginger Gingerbread,2002090414,rdsxc,PT35M,PT1H,PT1H35M,2020-12-22 15:27:00+00:00,Make and share this Meg's Fresh Ginger Gingerb...,[],...,54.4,278.2,48.5,0.8,22.8,3.9,8.0,1 8x8 cake pan,[Preheat oven to 350&deg;F Grease an 8x8 cake ...,None
522513,541380,Roast Prime Rib au Poivre with Mixed Peppercorns,211566,Denver cooks,PT3H,PT30M,PT3H30M,2020-12-22 15:32:00+00:00,"White, black, green, and pink peppercorns add ...",[https://img.sndimg.com/food/image/upload/w_55...,...,433.8,766.3,3.2,0.7,0.1,117.0,8.0,1 Roast,[Position rack in center of oven and preheat t...,None
522514,541381,Kirshwasser Ice Cream,2001131545,Jonathan F.,PT3H,PT1H,PT4H,2020-12-22 15:33:00+00:00,Make and share this Kirshwasser Ice Cream reci...,[],...,470.9,192.5,33.9,0.0,17.3,12.8,6.0,0,[heat half and half and heavy cream to a simme...,None
522515,541382,Quick & Easy Asian Cucumber Salmon Rolls,2001004241,CLUBFOODY,,PT15M,PT15M,2020-12-22 22:11:00+00:00,"Extremely quick and easy to make, these are gr...",[https://img.sndimg.com/food/image/upload/w_55...,...,2.9,100.5,0.3,0.0,0.1,2.4,0.0,20 rolls,"[In a small bowl, combine mayo and wasabi past...",None


In [16]:
cleaned_df = cleaned_df.iloc[:30]

In [17]:
cleaned_df.to_csv('./resource/limit30_cleaned_recipes.csv', index=False)

In [18]:
recipe_df = cleaned_df

In [19]:
from flask import Flask, jsonify, request
from flask_cors import CORS, cross_origin

app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'


@app.route('/recipes', methods=['GET'])
@cross_origin()
def get_recipes():
    return jsonify(recipe_df.to_dict(orient='records'))

@app.route('/recipes/<int:RecipeId>', methods=['GET'])
@cross_origin()
def get_recipe_by_id(RecipeId):
    recipe = recipe_df.loc[RecipeId]
    if recipe is None:
        return jsonify({"error": "Recipe not found"}), 404
    return jsonify(recipe.to_dict())

@app.route('/recipes?page<int:page>&pageSize<int:pageSize>', methods=['GET'])
@cross_origin()
def get_recipes_paginated(page, pageSize):
    return jsonify(recipe_df.loc[page:page+pageSize].to_dict(orient='records'))


In [38]:
app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
[2024-03-17 19:53:46,605] ERROR in app: Exception on /recipes/1 [GET]
Traceback (most recent call last):
  File "c:\Users\ASUS\anaconda3\Lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ASUS\anaconda3\Lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ASUS\anaconda3\Lib\site-packages\flask_cors\extension.py", line 176, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
                                                ^^^^^^^^^^^^^^^^^^
  File "c:\Users\ASUS\anaconda3\Lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ASUS\anaconda3\Lib\site-packages\flask\app.py", line 1796